Mount Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/My Drive/MLDM Project/data.zip"

!unzip '/content/drive/My Drive/MLDM Project/newDataset.zip'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Archive:  /content/drive/My Drive/MLDM Project/data.zip
  inflating: validation.hdf5         
  inflating: train.hdf5              
Archive:  /content/drive/My Drive/MLDM Project/newDataset.zip
  inflating: data.h5                 
  inflating: validation.h5           


In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM,CuDNNLSTM, Conv1D, MaxPooling1D,UpSampling1D, Input, Lambda, AveragePooling1D, GlobalAveragePooling1D
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
import numpy as np
from keras import backend as K
from keras.models import Model
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope


In [0]:
train_file_name = r"/content/data.h5"
test_file_name = r"/content/validation.h5"
labels = pd.read_csv("/content/drive/My Drive/MLDM Project/labels.csv", sep=",")


time_series = pd.read_hdf(train_file_name)
test_time_series = pd.read_hdf(test_file_name)
# time_series.iloc[0].index = time_series.iloc[0].index.astype(dtype='int64', copy=False)
labels = labels["anomaly"].to_numpy().astype(int)
labels = labels.reshape(-1, 1)

X_train = time_series
X_test = test_time_series

scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [0]:
'''
Based on work by Xianshun Chen
https://github.com/chen0040/keras-anomaly-detection
'''

from keras.layers import Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D,  Dense, Flatten, Input
from keras.models import Sequential
from keras.models import Model, model_from_json
from keras.callbacks import ModelCheckpoint
import numpy as np


class Conv1DAutoEncoder(object):
    model_name = 'con1d-auto-encoder'
    VERBOSE = 1

    def __init__(self):
        self.model = None
        self.encoder = None
        self.decoder = None
        self.time_window_size = None
        self.metric = None
        self.threshold = 5.0
        self.config = None

    @staticmethod
    def create_model(time_window_size, metric):
        # model = Sequential()
        # model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu',
        #                  input_shape=(time_window_size, 1)))
        # model.add(GlobalMaxPool1D())
        #
        # model.add(Dense(units=time_window_size, activation='linear'))

        input_layer = Input(shape=(time_window_size, 1))
        conv1 = Conv1D(filters=256, kernel_size=5, padding='same', activation='relu')(input_layer)
        pool1 = AveragePooling1D()(conv1)
        conv2 = Conv1D(filters=125, kernel_size=5, padding='same', activation='relu')(pool1)

        encoder = GlobalMaxPool1D()(conv2)
        # encoder = GlobalAveragePooling1D()(conv1)
        decoder = Dense(units=time_window_size, activation='linear')(encoder)
        model_encoder = Model(inputs=input_layer, outputs=encoder)
        # Input(shape=(encoding_dim,))
        # model_decoder = Model(inputs=dim, outputs=decoder)
        model = Model(inputs=input_layer, outputs=decoder)
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=[metric])
        print(model.summary())
        return model, model_encoder, decoder

    @staticmethod
    def get_config_file(model_dir_path):
        return model_dir_path + '/' + Conv1DAutoEncoder.model_name + '-config.npy'

    @staticmethod
    def get_weight_file(model_dir_path):
        return model_dir_path + '/' + Conv1DAutoEncoder.model_name + '-weights.h5'

    @staticmethod
    def get_architecture_file(model_dir_path):
        return model_dir_path + '/' + Conv1DAutoEncoder.model_name + '-architecture.json'

    def load_model(self, model_dir_path):
        config_file_path = self.get_config_file(model_dir_path)
        self.config = np.load(config_file_path).item()
        self.metric = self.config['metric']
        self.time_window_size = self.config['time_window_size']
        self.threshold = self.config['threshold']
        self.model, self.encoder, self.decoder = self.create_model(self.time_window_size, self.metric)
        # print('self.model: ', self.model)
        weight_file_path = self.get_weight_file(model_dir_path)
        self.model.load_weights(weight_file_path)

    def fit(self, dataset, model_dir_path, batch_size=8, epochs=100, validation_split=0.1, metric='mean_absolute_error',
            estimated_negative_sample_ratio=0.9):
        self.time_window_size = dataset.shape[1]
        self.metric = metric

        input_timeseries_dataset = np.expand_dims(dataset, axis=2)

        weight_file_path = self.get_weight_file(model_dir_path=model_dir_path)
        architecture_file_path = self.get_architecture_file(model_dir_path)
        checkpoint = ModelCheckpoint(weight_file_path)
        self.model, self.encoder, self.decoder = self.create_model(self.time_window_size, metric=self.metric)
        open(architecture_file_path, 'w').write(self.model.to_json())
        history = self.model.fit(x=input_timeseries_dataset, y=dataset,
                                 batch_size=batch_size, epochs=epochs,
                                 verbose=self.VERBOSE, validation_split=validation_split,
                                 callbacks=[checkpoint]).history
        self.model.save_weights(weight_file_path)

        scores = self.predict(dataset)
        scores.sort()
        cut_point = int(estimated_negative_sample_ratio * len(scores))
        self.threshold = scores[cut_point]

        print('estimated threshold is ' + str(self.threshold))

        self.config = dict()
        self.config['time_window_size'] = self.time_window_size
        self.config['metric'] = self.metric
        self.config['threshold'] = self.threshold
        config_file_path = self.get_config_file(model_dir_path=model_dir_path)
        np.save(config_file_path, self.config)

        return history

    def predict(self, timeseries_dataset):
        input_timeseries_dataset = np.expand_dims(timeseries_dataset, axis=2)
        target_timeseries_dataset = self.model.predict(x=input_timeseries_dataset)
        dist = np.linalg.norm(timeseries_dataset - target_timeseries_dataset, axis=-1)
        return dist

    def anomaly(self, timeseries_dataset, threshold=None):
        if threshold is not None:
            self.threshold = threshold

        dist = self.predict(timeseries_dataset)
        return zip(dist >= self.threshold, dist)


In [0]:
data_dir_path = './data'
model_dir_path = './models'

ae = Conv1DAutoEncoder()

ae.fit(X_train, model_dir_path=model_dir_path, estimated_negative_sample_ratio=0.99, epochs=1)

# load back the model saved in model_dir_path detect anomaly
# ae.load_model(model_dir_path)
# anomaly_information = ae.anomaly(ecg_np_data[:23, :])
# reconstruction_error = []
# for idx, (is_anomaly, dist) in enumerate(anomaly_information):
#     print('# ' + str(idx) + ' is ' + ('abnormal' if is_anomaly else 'normal') + ' (dist: ' + str(dist) + ')')
#     reconstruction_error.append(dist)

# visualize_reconstruction_error(reconstruction_error, ae.threshold)


Model: "model_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 12288, 1)          0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 12288, 256)        1536      
_________________________________________________________________
average_pooling1d_5 (Average (None, 6144, 256)         0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 6144, 125)         160125    
_________________________________________________________________
global_max_pooling1d_16 (Glo (None, 125)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 12288)             1548288   
Total params: 1,709,949
Trainable params: 1,709,949
Non-trainable params: 0
________________________________________________

{'loss': [0.004476935550980981],
 'mean_absolute_error': [0.03528853397635734],
 'val_loss': [0.0008519116360488813],
 'val_mean_absolute_error': [0.019789328362406437]}

In [0]:
ae.fit(X_train, model_dir_path=model_dir_path, estimated_negative_sample_ratio=0.99, epochs=4)


Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 12288, 1)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 12288, 256)        1536      
_________________________________________________________________
global_average_pooling1d_4 ( (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 12288)             3158016   
Total params: 3,159,552
Trainable params: 3,159,552
Non-trainable params: 0
_________________________________________________________________
None
Train on 7546 samples, validate on 839 samples
Epoch 1/4
7546/7546 [==============================] - 241s 32ms/step - loss: 0.8968 - mean_absolute_error: 0.5412 - val_loss: 0.3632 - val_mean_absolute_error: 0.3542
Epoch 2/4
7546/7546 [=====

{'loss': [0.8967701248960851,
  0.8880672248200336,
  0.8838524784613376,
  0.8826933048572713],
 'mean_absolute_error': [0.5412222944849718,
  0.5356636170743159,
  0.5336996646607362,
  0.5332512512167221],
 'val_loss': [0.36324655575426884,
  0.35849148620261906,
  0.35721674406095116,
  0.3561796826111759],
 'val_mean_absolute_error': [0.35421823487307375,
  0.351147339342959,
  0.35012875504445407,
  0.34928929481624277]}

In [0]:
print(ae.encoder.summary())

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 12288, 1)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 12288, 256)        1536      
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 256)               0         
Total params: 1,536
Trainable params: 1,536
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
target_timeseries_dataset = ae.anomaly(X_test)
# dist = np.linalg.norm(X_test - target_timeseries_dataset, axis=-1)

Ypred = []
reconstruction_error = []
for idx, (is_anomaly, dist) in enumerate(target_timeseries_dataset):
      predicted_label = 1 if is_anomaly else 0
      Ypred.append(predicted_label)
      reconstruction_error.append(dist)

error_sum = []
labels = []
prev = 0

for i in range(len(reconstruction_error) // 5):
    sub_distance = reconstruction_error[prev:prev + 5]
    sub_labels = Ypred[prev:prev + 5]
    prev = prev + 5
    error_sum.append(np.sum(sub_distance))
    labels.append(np.max(sub_labels))


# visualize_reconstruction_error(reconstruction_error, ae.threshold)
np.savetxt(r"distances.csv", np.c_[error_sum,labels], comments='',
           header="error,label",
           fmt='%f', delimiter=",")

In [0]:
train_predicted = ae.anomaly(X_train)
test_predicted = ae.anomaly(X_test)

In [0]:
y_train = list(train_predicted)
y_test = list(test_predicted)
dist_train = [x[1] for x in y_train]
dist_test = [x[1] for x in y_test]
# print("dist_train: ", train_predicted)

dist_train:  <zip object at 0x7f6114a10a08>


In [0]:
clf = OneClassSVM(nu=0.25, gamma="auto")

In [0]:
# print(train_predicted.shape)
clf.fit(np.array(dist_train).reshape(-1,1))

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
            max_iter=-1, nu=0.25, random_state=None, shrinking=True, tol=0.001,
            verbose=False)

In [0]:
y_pred_test = clf.predict(np.array(dist_test).reshape(-1,1))
y_pred_test[y_pred_test == 1] = 0
y_pred_test[y_pred_test == -1] = 1


In [0]:
error_sum = []
labels = []
prev = 0

for i in range(len(y_pred_test) // 5):
    sub_labels = y_pred_test[prev:prev + 5]
    prev = prev + 5
    # error_sum.append(np.sum(sub_distance))
    labels.append(np.max(sub_labels))


np.savetxt(r"svd_res.csv", np.c_[labels], comments='',
           header="anomaly",
           fmt='%f', delimiter=",")

In [0]:
#new Features
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(-1,X_train.shape[1],1)
X_test = X_test.reshape(-1,X_test.shape[1],1)
print(X_train.shape)
X_train_encoded = ae.encoder.predict(X_train)
X_test_encoded = ae.encoder.predict(X_test)

(8385, 12288, 1)


In [0]:
clf = IsolationForest(n_estimators=100,   behaviour='new',
                      contamination='auto', verbose=1, n_jobs=6)
# clf =EllipticEnvelope()
# clf = OneClassSVM(nu=0.3, gamma="auto")
clf.fit(X_train_encoded)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    0.4s remaining:    0.8s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.4s finished


IsolationForest(behaviour='new', bootstrap=False, contamination='auto',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=6, random_state=None, verbose=1, warm_start=False)

In [0]:
y_pred_test = clf.predict(X_test_encoded)
y_pred_test[y_pred_test == 1] = 0
y_pred_test[y_pred_test == -1] = 1


In [0]:
error_sum = []
labels = []
prev = 0

for i in range(len(y_pred_test) // 5):
    sub_labels = y_pred_test[prev:prev + 5]
    prev = prev + 5
    # error_sum.append(np.sum(sub_distance))
    labels.append(np.max(sub_labels))


np.savetxt(r"encoded_IF_res.csv", np.c_[labels], comments='',
           header="anomaly",
           fmt='%f', delimiter=",")

In [0]:
from sklearn.metrics import classification_report
truth =  pd.read_csv("/content/drive/My Drive/MLDM Project/labels.csv", sep=",")
truth = truth["anomaly"].to_numpy().astype(int)
truth = truth.reshape(-1, 1)

print("results: ", classification_report(labels, truth))

results:                precision    recall  f1-score   support

           0       0.98      0.89      0.93       327
           1       0.88      0.98      0.93       267

    accuracy                           0.93       594
   macro avg       0.93      0.93      0.93       594
weighted avg       0.93      0.93      0.93       594

